

### Namen en studenten nummers:

 **Murielle Tichelaar** (500926485)  
 **Nalini Bisessar** (500874334)  
 **Lushan Strack** (500934278)  
 **Dinand Voogt** (500934202) 


### Projectinformatie:

**Opleiding:** Toegepaste Wiskunde & Data Science  
**Vak:** Beroepsproject 3.4  
**Begeleider:** R. Nolet  
**Datum:** 20-02-2026  


### Doel/opdracht van het bestand:
In dit deployment script wordt een getraind model gebruikt om voorspellingen te doen op nieuwe klantdata.

Als input heeft dit script 2 bestanden, namelijk churndata_deployment.csv en het getrainde pipeline-bestand.

Vervolgens wordt voor iedere klant de kans op churn voorspeld. Op basis van deze kans en een gepersonaliseerde drempelwaarde wordt bepaald of tarief 2 wordt aangeboden, daarmee wordt ook de verwachte maandelijkse winst uitgerekend.

Als output wordt een CSV-bestand gecreeërd met 300 beslissingen (0/1), met of de klant tarief 2 krijgt of niet. Daarnaast wordt geprint hoeveel mensen tarief 2 krijgen aangeboden en de verwachte maandelijke winst wordt geprint.

In [1]:
import pandas as pd
import joblib
import numpy as np

In [2]:
# Pipeline laden
model_pipeline = joblib.load("../../data/model_pipeline.joblib")

In [3]:
# Data inladen
df_deploy = pd.read_csv("../../data/churndata_deployment.csv", sep=",", decimal=".")

X_deploy = df_deploy.copy()

In [4]:
# Kans voorspellen
y_prob = model_pipeline.predict_proba(X_deploy)[:, 1]

In [5]:
# Offer tarrief 2 berekenen

# Winstvergelijkingen
W1 = 0.1 * X_deploy["Frequency of SMS"] + 0.3 * X_deploy["Seconds of Use"]
W2 = 0.07 * X_deploy["Frequency of SMS"] + 0.2 * X_deploy["Seconds of Use"]

thresholds = (W1 - W2) / (W1 - 0.75 * W2)

# Beslissing
offer_tarief2 = (y_prob >= thresholds).astype(int)

In [6]:
# Output wegschrijven
output = pd.DataFrame({
    "Tarief2_aanbieden": offer_tarief2
})

output.to_csv("../../data/deployment_output.csv", index=False)

In [ ]:
# Winst omzetten naar per maand
sms_pm = X_deploy["Frequency of SMS"] / 9
min_pm = (X_deploy["Seconds of Use"] / 60) / 9  # seconden -> minuten -> per maand

W1_pm = 0.1 * sms_pm + 0.3 * min_pm
W2_pm = 0.07 * sms_pm + 0.2 * min_pm

# Verwachte winst per klant per maand
E_tarief1 = (1 - y_prob) * W1_pm
E_tarief2 = (1 - 0.75 * y_prob) * W2_pm

# Totale verwachte winst 
totale_winst = E_tarief2[offer_tarief2 == 1].sum() + E_tarief1[offer_tarief2 == 0].sum()


print("Aantal klanten tarief 2:", int(offer_tarief2.sum()))
print("Geschatte maandelijkse winst:", round(float(totale_winst), 2))

Aantal klanten tarief 2: 18
Geschatte maandelijkse winst: 963.03
